In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data reading

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [ ]:
Data=pd.read_excel('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/new data/hasoc_2020_en_train_new.xlsx' ,sep='\t')
Data.head()

#Data Preprocessing
Stop Word Removal

In [ ]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
Data['text'] = Data['text'].apply(lambda x: remove_sw(x))
Data[['text','task2']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,task2
0,"hate wen females hit ah nigga tht bro 😂😂, I’m tryna make u la sweety , fuck ah bro",PRFN
1,RT @airjunebug: When Bay really NY nigga heart. W/ @supportcaleon https://t.co/mZ8BAYlnlf,PRFN
2,"RT @DonaldJTrumpJr: Dear Democrats: The American people aren’t stupid, know spying amount gaslig...",NONE
3,RT @SheLoveTimothy: He ain’t drugs bored. I shit I’m bored 😂 https://t.co/tkdjSbddET,PRFN
4,"RT @TavianJordan: Summer ‘19 I’m coming ! No boring shit ! Beach days, road trips, kickbacks HOT...",NONE


Removing Emoji

In [ ]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['text']=Data['text'].apply(lambda x:deEmojify(x))
Data[['text','task2']].head()

,text,task2
0,"hate wen females hit ah nigga tht bro , I’m tryna make u la sweety , fuck ah bro",PRFN
1,RT @airjunebug: When Bay really NY nigga heart. W/ @supportcaleon https://t.co/mZ8BAYlnlf,PRFN
2,"RT @DonaldJTrumpJr: Dear Democrats: The American people aren’t stupid, know spying amount gaslig...",NONE
3,RT @SheLoveTimothy: He ain’t drugs bored. I shit I’m bored https://t.co/tkdjSbddET,PRFN
4,"RT @TavianJordan: Summer ‘19 I’m coming ! No boring shit ! Beach days, road trips, kickbacks HOT...",NONE


Removing URL

In [ ]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data[['text']].head()

Data['text']=Data['text'].str.replace(r'\d+','')
Data[['text','task2']].head()

,text,task2
0,"hate wen females hit ah nigga tht bro , I’m tryna make u la sweety , fuck ah bro",PRFN
1,RT @airjunebug: When Bay really NY nigga heart. W/ @supportcaleon,PRFN
2,"RT @DonaldJTrumpJr: Dear Democrats: The American people aren’t stupid, know spying amount gaslig...",NONE
3,RT @SheLoveTimothy: He ain’t drugs bored. I shit I’m bored,PRFN
4,"RT @TavianJordan: Summer ‘ I’m coming ! No boring shit ! Beach days, road trips, kickbacks HOT D...",NONE


# Tokenization, Encoding, padding after preprocessing

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [ ]:
tok1.fit_on_texts(Data['text'])
words=len(tok1.word_counts)
words

11848

In [ ]:
word_list=[tok1.word_index]
#word_list

In [ ]:
encoded=tok1.texts_to_sequences(Data['text'])
#print(Data['text'][0])
encoded[1211]

[1, 2769, 2770, 6195, 2771, 1768, 984, 404, 198, 63, 69, 16, 6196, 6197]

In [ ]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
padd[0:12]

array([[  98, 1496, 1193,  765,  212,  160, 3355,  283,   17,  335,   89,
          33,  600, 3356,    3,  212,  283,    0,    0,    0],
       [   1, 3357,  121, 3358,   44, 1497,  160,  376,  173, 3359,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   1, 1498,  766,  601,   12,  602,   14, 1194,   40,   18, 2059,
        1001, 3360,  677,  199,    0,    0,    0,    0,    0],
       [   1, 3361,   69,  251, 1002, 2060,    2,    5,   17, 2060,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [3362, 3363, 1499,   17,  127,   58,  767,    5, 2061,  161,  678,
        3364, 3365,  336,  161,   17,  252,   17,  252, 2062],
       [3366,  399,    5,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   1, 3367,   18,    3,  603, 3368, 3369, 1195,  213,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [2063, 3370,  461,  106, 3371, 119

# Label Encoding

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['task2'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(Data['task2'])
train_label=to_categorical(np.asarray(train_labelEncode))

['HATE' 'NONE' 'OFFN' 'PRFN']


# Reading embedding vector using word2vec

In [ ]:
embeddings_index = {}
f = open('drive/My Drive/Summer Internship 2020 July/My Test File/Sunil/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((words+1, 300))
for word, i in tok1.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

# Importing libraries for Model creation

In [ ]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM

# Preparing Embedding layer

In [ ]:
Embedding_Layer = Embedding(words+1,300,weights=[embedding_matrix], input_length=max_length, trainable=False)

#LSTM model

In [ ]:
model = Sequential()
model.add(Embedding_Layer)
model.add(LSTM(256,return_sequences=False))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
#model.add(Dense(16, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           3554700   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               570368    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 4,141,776
Trainable params: 587,076
Non-trainable params: 3,554,700
_________________________________________________________________


# Model Compilation

In [ ]:
## compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Model Training

In [ ]:
model.fit(padd,train_label,epochs=25,verbose=1,batch_size=16)

Epoch 1/25
232/232 [==============================] - 1s 6ms/step - loss: 0.3289 - acc: 0.7441
Epoch 2/25
232/232 [==============================] - 1s 6ms/step - loss: 0.2691 - acc: 0.8074
Epoch 3/25
232/232 [==============================] - 1s 6ms/step - loss: 0.2478 - acc: 0.8207
Epoch 4/25
232/232 [==============================] - 1s 6ms/step - loss: 0.2341 - acc: 0.8328
Epoch 5/25
232/232 [==============================] - 1s 5ms/step - loss: 0.2177 - acc: 0.8390
Epoch 6/25
232/232 [==============================] - 1s 5ms/step - loss: 0.2013 - acc: 0.8460
Epoch 7/25
232/232 [==============================] - 1s 6ms/step - loss: 0.1816 - acc: 0.8689
Epoch 8/25
232/232 [==============================] - 1s 6ms/step - loss: 0.1608 - acc: 0.8873
Epoch 9/25
232/232 [==============================] - 1s 6ms/step - loss: 0.1421 - acc: 0.9051
Epoch 10/25
232/232 [==============================] - 1s 6ms/step - loss: 0.1253 - acc: 0.9199
Epoch 11/25
232/232 [============================

#Test data reading

In [ ]:
Test =pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/test data/english_test_1509.csv')
Test.dropna(inplace = True)
Test.head()

,tweet_id,text,task1,task2,ID
0,1130081762154090497,RT @delmiyaa: Samini resetting the show and moving things along as if nothing happened is all yo...,NOT,NONE,hasoc_2020_en_2713
1,1130048316807491584,@Swxnsea how do you know that he’s left?,HOF,NONE,hasoc_2020_en_3874
2,1123657766143504386,"Tried to get Divock Origi on a free seeing as he was at the club on loan. He accepted the offer,...",NOT,NONE,hasoc_2020_en_281
3,1126782963042013186,RT @nutclusteruwu: that....is yalls stupid white ass reactions meeting tom holland in disneyland...,HOF,PRFN,hasoc_2020_en_2026
4,1130159113529434113,&amp; IT DID. But a bitch got big girls things to do now 😇,HOF,PRFN,hasoc_2020_en_4023


# Test data preprocessing

In [ ]:
## doing preprocessing as above

Test['text'] = Test['text'].apply(lambda x: remove_sw(x))
Test['text']=Test['text'].apply(lambda x:deEmojify(x))
Test['text']=Test['text'].apply(lambda x: remove_URL(x))
Test['text']=Test['text'].str.replace(r'\d+','')
t= Test['text']

# Test data encoding, padding

In [ ]:
encoded1 =tok1.texts_to_sequences(t)
print(t[0])
encoded1[0]

RT @delmiyaa: Samini resetting show moving things along nothing happened need know greatness.


[1, 107, 1762, 256, 642, 265, 901, 16, 18]

In [ ]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([   1,  107, 1762,  256,  642,  265,  901,   16,   18,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

# Test label encoding

In [ ]:
test_labelEncode=labelEncode.transform(Test['task2'])
test_label=to_categorical(np.asarray(test_labelEncode))

# Test data Model prediction

In [ ]:
test_predictions=model.predict(padded)
test_predictions2=np.argmax(test_predictions,axis=1)

# Test data Classification report

In [ ]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_labelEncode,test_predictions2))

              precision    recall  f1-score   support

           0       0.23      0.20      0.21        25
           1       0.86      0.94      0.90       414
           2       0.57      0.15      0.23        82
           3       0.81      0.87      0.84       293

    accuracy                           0.81       814
   macro avg       0.62      0.54      0.55       814
weighted avg       0.79      0.81      0.79       814



# Adding predicted lable as a colum to dataFrame 'Test'

In [ ]:
Test['Predictionlabel']=labelEncode.inverse_transform(test_predictions2)
Test.head()

,tweet_id,text,task1,task2,ID,Predictionlabel
0,1130081762154090497,RT @delmiyaa: Samini resetting show moving things along nothing happened need know greatness.,NOT,NONE,hasoc_2020_en_2713,NONE
1,1130048316807491584,@Swxnsea know he’s left?,HOF,NONE,hasoc_2020_en_3874,NONE
2,1123657766143504386,"Tried get Divock Origi free seeing club loan. He accepted offer, actual…",NOT,NONE,hasoc_2020_en_281,NONE
3,1126782963042013186,RT @nutclusteruwu: that....is yalls stupid white ass reactions meeting tom holland disneyland? f...,HOF,PRFN,hasoc_2020_en_2026,PRFN
4,1130159113529434113,&amp; IT DID. But bitch got big girls things,HOF,PRFN,hasoc_2020_en_4023,PRFN


# Generating CSV file

In [ ]:
EnglishTask1File=Test[['tweet_id','Predictionlabel','ID']]
EnglishTask1File.head()
EnglishTask1File.to_csv('submision_EN_B.csv',index=False)